In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from itertools import combinations

In [2]:
from utils import similarity_matrix

In [3]:
def plot_all_pair_similarities(df, embeddings_by_method, window_size=5):
    pair_list = []

    # Collect unique unordered protein pairs per comparison group
    grouped = df.groupby('PAIR')
    for comparison_id, group in grouped:
        proteins = group['EMBEDDINGid'].unique()
        for id1, id2 in combinations(proteins, 2):
            pair_list.append((id1, id2, comparison_id))

    methods = list(embeddings_by_method.keys())
    nrows = len(pair_list)
    ncols = len(methods)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 3 * nrows), dpi=500)

    # Ensure axes is always 2D for consistent indexing
    if nrows == 1:
        axes = axes[np.newaxis, :]
    if ncols == 1:
        axes = axes[:, np.newaxis]

    for row_idx, (id1, id2, comparison_id) in enumerate(pair_list):
        for col_idx, method in enumerate(methods):
            protein_embeddings = embeddings_by_method[method]

            try:
                emb1 = protein_embeddings[id1]
                emb2 = protein_embeddings[id2]
                sim_matrix, (best_emb1, best_emb2, best_score) = similarity_matrix(emb1, emb2, window_size)
            except ValueError as e:
                print(f"Skipping {id1}-{id2} due to error: {e}")
                continue

            ax = axes[row_idx, col_idx]
            im = ax.imshow(sim_matrix, aspect='auto', cmap='viridis')

            # Set shared row label (pair name) on the first column
            if col_idx == 0:
                ax.set_ylabel(f"{comparison_id}", fontsize=20, labelpad=10)

            # Set individual method title
            if row_idx == 0:
                ax.set_title(method, fontsize=24)
                
            ax.set_xticks([])
            ax.set_yticks([])

    plt.tight_layout()
    plt.show()


In [4]:
embeddings_by_method = {
    'ProtT5': pickle.load(open("../embeddings/prott5_aligned_embs.pkl", "rb")),
    'ESM-2(3B)': pickle.load(open("../embeddings/esm2_3B_aligned_embs.pkl", "rb")),
    'ESM-2(15B)': pickle.load(open("../embeddings/esm2_15B_aligned_embs.pkl", "rb")),
    'CARP': pickle.load(open("../embeddings/carp_aligned_embs.pkl", "rb")),
    'ProstT5': pickle.load(open("../embeddings/prostt5_aligned_embs.pkl", "rb"))
}
df = pd.read_csv("../datasets/malisam_sequences_aligned.csv")
df["EMBEDDINGid"] = df["SCOPid"].astype(str) + '_' + df.index.astype(str)

In [5]:
def shuffle_in_pairs(df, random_state=None):
    # number of pairs
    n_pairs = len(df) // 2  
    
    # create pair indices: (0,1), (2,3), ...
    pairs = [(2*i, 2*i+1) for i in range(n_pairs)]
    
    # shuffle the order of pairs
    rng = np.random.default_rng(random_state)
    rng.shuffle(pairs)
    
    # flatten the shuffled pairs back into row indices
    new_order = [i for pair in pairs for i in pair]
    
    return df.iloc[new_order].reset_index(drop=True)

In [6]:
df.head()

,COMPARISON,PAIR,SCOPid,RANGE,SCOP CLASS,SCOP SUPERFAMILY,EMBEDDINGid,SEQUENCE,SEQUENCElen
0,1.0,d1a2za_d1ghha_,d1a2za_,"A74-A81,A114-A138,A159-A220",a/b,Pyrrolidone carboxyl peptidase (pyroglutamate ...,d1a2za__0,YSNITVERATLPVRAITKTLRDNGIPATISYSAYPLKAGFIHVPYT...,95
1,1.0,d1a2za_d1ghha_,d1ghha_,"A1-A12,A13-A46,A47-A81",a+b,DNA damage-inducible protein DinI,d1ghha__1,MRIEVTIAKTSPLPAGAIDALAGELSRRIQYAFPDNEGHVSVRYAA...,81
2,2.0,d1a2za_d1u9da_,d1a2za_,"A75-A81,A114-A138,A159-A220,A61-A74",a/b,Pyrrolidone carboxyl peptidase (pyroglutamate ...,d1a2za__2,SNITVERATLPVRAITKTLRDNGIPATISYSAYPLKAGFIHVPYTP...,108
3,2.0,d1a2za_d1u9da_,d1u9da_,"A1-A9,A10-A48,A49-A84,A85-A107",a+b,Tautomerase/MIF,d1u9da__3,MPHLRFRAVEAHIVESLVPTLLNELSSLLSTARNAFTFELINTQYF...,107
4,3.0,d1a7j__d1kafa_,d1a7j__,203-279,a/b,P-loop containing nucleoside triphosphate hydr...,d1a7j___4,DINFQRVPVVDTSNPFIARWIPTADESVVVIRFRNPRGIDFPYLTS...,77


In [7]:
len(df)

260

In [8]:
plot_all_pair_similarities(shuffle_in_pairs(df, random_state=42)[:10], embeddings_by_method, window_size=5)